# Foundations of Computer Science Project - Kiva Crowdfunding

In [1]:
import numpy as np, pandas as pd

### 1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [2]:
ll = pd.read_csv('additional-kiva-snapshot/loans_lenders.csv', nrows=5)
ll.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [3]:
#dataframe gets huge after splitting, declaring data types to optimize memory management
ll = pd.read_csv('additional-kiva-snapshot/loans_lenders.csv', dtype={'loan_id': np.uint32})

loans = pd.Series(ll['loan_id'])
#splitting each lenders list into separate names
lenders = ll['lenders'].str.split(',').explode()
ll = pd.DataFrame({'loan_id': loans, 'lender': lenders})
del loans, lenders

ll.head()

,loan_id,lender
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499


### 2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [4]:
loans = pd.read_csv('additional-kiva-snapshot/loans.csv', nrows=5)
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,True,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,True,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,True,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,True,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,True,bullet,field_partner


In [5]:
#big dataset: unused attributes not imported and data types declared in order to save space
loans.columns

Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'activity_name', 'sector_name', 'loan_use', 'country_code',
       'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders',
       'borrower_pictured', 'repayment_interval', 'distribution_model'],
      dtype='object')

In [6]:
loans = pd.read_csv('additional-kiva-snapshot/loans.csv',
                    usecols=['loan_id', 'funded_amount', 'loan_amount', 'status', 'country_code', 'country_name', 
                             'planned_expiration_time', 'disburse_time', 'num_lenders_total', 'repayment_interval'],
                    dtype={'loan_id': np.uint32, 'funded_amount': np.uint32, 'loan_amount': np.uint32,
                           'status': 'category', 'country_code': 'category', 'country_name': 'category',
                          'planned_expiration_time': 'str', 'disburse_time': 'str',
                           'num_lenders_total': np.uint32, 'repayment_interval': 'category'})
loans.head()

,loan_id,funded_amount,loan_amount,status,country_code,country_name,planned_expiration_time,disburse_time,num_lenders_total,repayment_interval
0,657307,125,125,funded,PH,Philippines,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,3,irregular
1,657259,400,400,funded,HN,Honduras,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,11,monthly
2,658010,400,400,funded,PK,Pakistan,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,16,monthly
3,659347,625,625,funded,KG,Kyrgyzstan,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,21,monthly
4,656933,425,425,funded,PH,Philippines,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,15,bullet


In [7]:
loans['disburse_time'].isnull().value_counts()

False    1416794
True        2813
Name: disburse_time, dtype: int64

In [8]:
loans['planned_expiration_time'].isnull().value_counts()

False    1047773
True      371834
Name: planned_expiration_time, dtype: int64

In [9]:
loans['duration'] = (loans['planned_expiration_time'].astype('datetime64') - loans['disburse_time'].astype('datetime64'))
loans.head()

,loan_id,funded_amount,loan_amount,status,country_code,country_name,planned_expiration_time,disburse_time,num_lenders_total,repayment_interval,duration
0,657307,125,125,funded,PH,Philippines,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,3,irregular,53 days 19:30:06
1,657259,400,400,funded,HN,Honduras,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,11,monthly,96 days 14:25:07
2,658010,400,400,funded,PK,Pakistan,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,16,monthly,37 days 13:10:05
3,659347,625,625,funded,KG,Kyrgyzstan,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,21,monthly,34 days 19:10:02
4,656933,425,425,funded,PH,Philippines,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,15,bullet,57 days 22:10:02


### 3. Find the lenders that have funded at least twice.

In [10]:
ll.groupby('lender').count().query('loan_id > 1')

,loan_id
lender,
000,39
00000,39
0002,70
0101craign0101,71
0132575,4
...,...
zyrorl,3
zzaman,11
zzanita,2


### 4. For each country, compute how many loans have involved that country as borrowers.

In [11]:
loans.groupby('country_name')['loan_id'].count()

country_name
Afghanistan     2337
Albania         3075
Armenia        13952
Azerbaijan     10172
Belize           218
               ...  
Canada             1
Botswana           1
Bhutan             2
Mauritania         1
Uruguay            1
Name: loan_id, Length: 96, dtype: int64

### 5. For each country, compute the overall amount of money borrowed.

In [12]:
loans.groupby('country_name')['loan_amount'].sum()

country_name
Afghanistan     1967950
Albania         4307350
Armenia        22950475
Azerbaijan     14784625
Belize           150175
                 ...   
Canada            50000
Botswana           8000
Bhutan            20000
Mauritania        15000
Uruguay            8000
Name: loan_amount, Length: 96, dtype: uint32

### 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [13]:
loans.groupby('country_name')['funded_amount'].sum() / loans.groupby('country_name')['loan_amount'].sum()

country_name
Afghanistan    0.998323
Albania        0.926898
Armenia        0.883846
Azerbaijan     0.959872
Belize         1.000000
                 ...   
Canada         1.000000
Botswana       1.000000
Bhutan         0.781250
Mauritania     1.000000
Uruguay        1.000000
Length: 96, dtype: float64

### 7. Like the three previous points, but split for each year (with respect to disburse time).

In [14]:
loans['disburse_year'] = loans['disburse_time'].str.split('-', expand= True)[0]
loans.groupby(['country_name','disburse_year'])['funded_amount'].sum()

country_name  disburse_year
Afghanistan   2005                  NaN
              2006                  NaN
              2007             194975.0
              2008             365375.0
              2009             581825.0
                                 ...   
Uruguay       2014                  NaN
              2015                  NaN
              2016                  NaN
              2017                  NaN
              2018               8000.0
Name: funded_amount, Length: 1344, dtype: float64

### 8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [15]:
ll = ll.join(loans[['loan_id','loan_amount','num_lenders_total']].set_index('loan_id'), on='loan_id')
ll.head()

,loan_id,lender,loan_amount,num_lenders_total
0,483693,muc888,1225.0,44.0
0,483693,sam4326,1225.0,44.0
0,483693,camaran3922,1225.0,44.0
0,483693,lachheb1865,1225.0,44.0
0,483693,rebecca3499,1225.0,44.0


In [16]:
ll['quota'] = ll['loan_amount'] / ll['num_lenders_total']
total_lent = pd.DataFrame(ll.groupby('lender', as_index=False)['quota'].sum()) #saved in a dataframe to be used in point 9
total_lent.columns = ['permanent_name', 'total_lent']
total_lent.head()

,permanent_name,total_lent
0,000,1485.309656
1,00000,1249.947363
2,0002,2201.180463
3,00mike00,38.461538
4,0101craign0101,2424.088932


### 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [17]:
lenders = pd.read_csv('additional-kiva-snapshot/lenders.csv', nrows=5)
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [18]:
#big dataset: unused attributes not imported and data types declared in order to save space
lenders = pd.read_csv('additional-kiva-snapshot/lenders.csv',
                    usecols=['permanent_name', 'country_code', 'occupation'],
                    dtype={'country_code': 'category', 'occupation': 'category'})
lenders.head()

,permanent_name,country_code,occupation
0,qian3013,NaN,NaN
1,reena6733,NaN,NaN
2,mai5982,NaN,NaN
3,andrew86079135,NaN,NaN
4,nguyen6962,NaN,NaN


In [19]:
lenders.groupby('country_code').size()

country_code
AE    1043
AL      96
AM      93
AR    4299
AS      24
      ... 
GF       5
PN       2
CX       3
IO       1
MS       1
Length: 234, dtype: int64

In [20]:
cc_dist=lenders['country_code'].value_counts(normalize= True)
cc_dist

US    0.664330
CA    0.076325
GB    0.043097
AU    0.041664
DE    0.017975
        ...   
TF    0.000001
IO    0.000001
NU    0.000001
GW    0.000001
MS    0.000001
Name: country_code, Length: 234, dtype: float64

In [21]:
nocountry = lenders['country_code'].isnull()
sample = lenders['country_code'].dropna().sample(nocountry.sum(), replace=True).values
lenders.loc[nocountry, 'country_code'] = sample
lenders.head()

,permanent_name,country_code,occupation
0,qian3013,SE,NaN
1,reena6733,US,NaN
2,mai5982,US,NaN
3,andrew86079135,US,NaN
4,nguyen6962,US,NaN


In [22]:
lenders = lenders.merge(total_lent, on='permanent_name')
lenders.head()

,permanent_name,country_code,occupation,total_lent
0,naresh2074,US,NaN,58.820663
1,jinxian2217,US,NaN,45.161290
2,sreedevi9583,US,NaN,25.000000
3,nicole98153893,US,NaN,28.571429
4,prashankpradeep8456,US,NaN,25.000000


In [23]:
country_lent = pd.DataFrame(lenders.groupby('country_code').sum())
country_borr = pd.DataFrame(loans[['country_code', 'loan_amount']].groupby('country_code').sum())
country_borr.head()
country_money = country_lent.merge(country_borr, on='country_code').reset_index()
country_money['net_lent'] = country_money['total_lent'] - country_money['loan_amount']
country_money[['country_code', 'net_lent']].head()

,country_code,net_lent
0,AL,-4.305598e+06
1,AM,-2.294887e+07
2,BG,-3.534498e+05
3,BR,-1.083375e+06
4,CA,4.640923e+06


### 10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [24]:
country_stats = pd.read_csv('additional-kiva-snapshot/country_stats.csv', nrows=5)
country_stats.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh


In [25]:
country_stats = pd.read_csv('additional-kiva-snapshot/country_stats.csv',
                            usecols=['country_code', 'population', 'population_below_poverty_line'])
country_stats = country_stats.merge(country_money, on='country_code')
country_stats['lent_per_capita'] = country_stats['net_lent'] / country_stats['population']
country_stats.head()

,country_code,population,population_below_poverty_line,total_lent,loan_amount,net_lent,lent_per_capita
0,IN,1339180127,21.9,133156.943798,8988200,-8.855043e+06,-0.006612
1,NG,190886311,70.0,9155.678877,7751725,-7.742569e+06,-0.040561
2,MX,129163276,46.2,72923.177271,25106300,-2.503338e+07,-0.193812
3,PK,197015955,29.5,11281.767877,24995850,-2.498457e+07,-0.126815
4,CD,81339988,63.0,56.818182,18813425,-1.881337e+07,-0.231293


In [26]:
country_stats.loc[country_stats['lent_per_capita'].idxmax(),['country_code', 'lent_per_capita']]

country_code             CA
lent_per_capita    0.126717
Name: 47, dtype: object

### 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [27]:
country_stats['lent_per_rich'] = country_stats['net_lent'] / ( country_stats['population'] * (1 - country_stats['population_below_poverty_line'] / 100 ) )
country_stats.head()

,country_code,population,population_below_poverty_line,total_lent,loan_amount,net_lent,lent_per_capita,lent_per_rich
0,IN,1339180127,21.9,133156.943798,8988200,-8.855043e+06,-0.006612,-0.008466
1,NG,190886311,70.0,9155.678877,7751725,-7.742569e+06,-0.040561,-0.135204
2,MX,129163276,46.2,72923.177271,25106300,-2.503338e+07,-0.193812,-0.360245
3,PK,197015955,29.5,11281.767877,24995850,-2.498457e+07,-0.126815,-0.179879
4,CD,81339988,63.0,56.818182,18813425,-1.881337e+07,-0.231293,-0.625116


In [28]:
country_stats.loc[country_stats['lent_per_rich'].idxmax(),['country_code', 'lent_per_rich']]

country_code           CA
lent_per_rich    0.139865
Name: 47, dtype: object

### 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [29]:
#the function takes in the start and end dates, and returns the amount of days intercurring between them, divided by year
def day_calc(date1, date2):
    (begin,end) = (date1,date2) if date1<date2 else (date2,date1)
    date_list = list()
    date_list.append(begin)
    for y in range(begin.year+1, end.year+1):
        date_list.append(pd.Timestamp(year=y, month=1, day=1, tz='utc'))
    date_list.append(end)
    
    res = dict()
    for i in range(len(date_list)-1):
        res[date_list[i].year] = (date_list[i+1] - date_list[i]).days

    return pd.Series(res, name='DateValue')

In [ ]:
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
loans['disburse_time'] = pd.to_datetime(loans['disburse_time'])
loans.head()

In [ ]:
loans = loans.dropna() #we kept NaNs until now but they break the function, so we drop them here
days = loans.apply(lambda x: day_calc(x['planned_expiration_time'],x['disburse_time']), axis = 1)
days.head()

In [ ]:
df_day_weighted.fillna(0, inplace= True)
df_day_weighted = pd.concat([df_day_weighted, data_loans['loan_id']
                 , data_loans['loan_amount']
                 , data_loans['duration'] ]
                , axis = 1, sort = True)

df_day_weighted.head()

In [ ]:
final = df_day_weighted[[2011,2012,2013,2014,2015,2016,2017,2018
                        ]].div(df_day_weighted.duration, axis=0).mul(df_day_weighted.loan_amount, axis = 0)

final = pd.concat([df_day_weighted['loan_id'], final],  axis = 1 )
final = final.melt(id_vars=["loan_id"], 
        var_name="Year", 
        value_name="amount_per_year")

final.groupby(['loan_id', 'Year']).sum()

### 13. For each value of repayment_interval, add a new column to the lenders dataframe that contains the total amount of money corresponding to loans in such state

In [ ]:
loans_repayment = loans.groupby(['repayment_interval', 'country_code'])
loans_repayment.head()

In [ ]:
lenders.head()

In [ ]:
for i in ['bullet', 'irregular', 'monthly', 'weekly']:
    lenders[i] = lenders.groupby('country_code')

In [ ]:
df13 = loans.groupby('country_code')['repayment_interval'].count()
df13.head()

In [ ]:
loans['repayment_interval'].head()

In [ ]:
ll.head()

In [ ]:
loans.columns

In [ ]:
def attach_totals_to_lenders():

    # {'country_code' := NaN} è considerato uno stato

    sum_by_country = loans.groupby('country_code', as_index=False).sum()[['country_code', 'funded_amount']]

    return pd.merge(ll, sum_by_country, left_on='country_code', right_on='country_code')

In [ ]:
out_13 = attach_totals_to_lenders()

display(out_13)

In [ ]:
df13 = df2.groupby('country_code', as_index=False).sum()[['country_code', 'funded_amount']]

In [ ]:
pd.merge(df13,df3, left_on='country_code', right_on='country_code')

### 14. What is the occupation with the highest average amount of money lent (the average must be computed over all lenders with a given occupation)?

In [ ]:
df3.occupation.head(50)

In [ ]:
df14_1=df3[['occupation','permanent_name']]

In [ ]:
df14_1.head()

In [ ]:
df14=df14_1.dropna()

In [ ]:
df14.head()

In [ ]:
df8_1.head()

In [ ]:
df14_3=pd.DataFrame(df8_1.groupby('Finanziatori').quota.sum())

In [ ]:
df14_3.head()

In [ ]:
df14_3=df14_3.reset_index()

In [ ]:
df14_3.head()

In [ ]:
df14 = df14.rename(columns={"occupation": "occupation", "permanent_name": "Finanziatori"})

In [ ]:
df14.head()

In [ ]:
df14_mix= df14.join(df14_3[['Finanziatori','quota']].set_index('Finanziatori'), on='Finanziatori')

In [ ]:
df14_mix.head(50)

In [ ]:
df14_mix.groupby('occupation')['quota'].sum().idxmax()


In [ ]:
df14_mix.groupby('occupation').sum()['quota'].idxmax()

### 15. Cluster the loans according to the year-month of disburse time.

In [ ]:
df2[['funded_amount','disburse_time']].head(30)

In [ ]:
df2.disburse_time.str.split('-', expand=True)

In [ ]:
df2['punto7'] = df2.disburse_time.str.split('-', expand= True)[0]

In [ ]:
df2['punto15'] = df2.disburse_time.str.split('-', expand= True)[1]

In [ ]:
df2.groupby(['punto7','punto15'])['funded_amount'].sum()

### 16. For each country, compute its overall GNI, by multiplying the per capita GNI with its population.

In [ ]:
df10.head()

In [ ]:
df10['overallGNI']= df10['population'] * df10['gni']

In [ ]:
df10.overallGNI.head(20)

### 17. Find the country with the highest rate of irregular repayment interval.

In [31]:
repayments = pd.DataFrame(loans.groupby(['country_code', 'repayment_interval'], as_index=False)['loan_id'].count())
repayments = repayments.merge(repayments.groupby('country_code')['loan_id'].sum(), on='country_code')
repayments = repayments[repayments['repayment_interval'] == 'irregular']
repayments['irreg_ratio'] = repayments['loan_id_x'] / repayments['loan_id_y']
repayments.loc[repayments['irreg_ratio'].idxmax(),['country_code', 'irreg_ratio']]

country_code    LK
irreg_ratio      1
Name: 149, dtype: object

### 18. Find the country with the highest fraction of loaned amount with irregular repayment interval.

In [30]:
irreg_loans = loans[loans['repayment_interval'] == 'irregular'].groupby('country_code', as_index=False)[['country_code', 'loan_amount']].sum()
irreg_loans = irreg_loans.rename(columns={'loan_amount': 'irreg_amount'})
irreg_loans = irreg_loans.merge(loans.groupby('country_code', as_index=False)['loan_amount'].sum())
irreg_loans['irreg_ratio'] = irreg_loans['irreg_amount'] / irreg_loans['loan_amount']
irreg_loans.loc[irreg_loans['irreg_ratio'].idxmax(),['country_code', 'irreg_ratio']]

country_code    LK
irreg_ratio      1
Name: 37, dtype: object